In [ ]:
import cv2
import numpy as np
from skimage.util import random_noise
from matplotlib import pyplot as plt
from skimage.metrics import peak_signal_noise_ratio
import torch
import glob
from IPython.display import clear_output

noiseL = 15
def process_image(image):
    image = torch.FloatTensor(image)/255
    return image

def add_noise(img):
    mean = torch.zeros(img.size(), dtype=torch.float32)
    noise = torch.normal(mean=mean, std=noiseL/255.)
    imgn_test = torch.clamp(img + noise, 0., 1.)
    return imgn_test

def output(original,noisy):
    cv2.imwrite("original.png", original.numpy()*255)
    cv2.imwrite("noisy.png", noisy.numpy()*255)
    
    
def compute_avg_psnr(gt_img, output_img, data_range=1):
    gt_img = gt_img.numpy().astype(np.float32)
    output_img = output_img.numpy().astype(np.float32)
    PSNR=0
    for i in range(gt_img.shape[0]):
        PSNR += peak_signal_noise_ratio(output_img[i,:,:], gt_img[i,:,:], data_range=data_range)
    return PSNR/gt_img.shape[0]

    
counter = 0
total_psnr = 0
total_psnr_denoised = 0
for filename in glob.glob('/content/drive/MyDrive/Datasets/val/*.PNG'):
    clear_output(wait=True)
    img = cv2.resize(cv2.imread(filename), (50,50), interpolation = cv2.INTER_AREA)
    img = process_image(img)
    noisy = add_noise(img)
    cv2.imwrite("noisy.png", noisy.numpy()*255)
    noisy_img = cv2.imread("noisy.png")
    denoised = cv2.fastNlMeansDenoisingColored(noisy_img,None,10,10,7,21)
    psnr = compute_avg_psnr(img,noisy)
    total_psnr+=psnr
    psnr_denoised = compute_avg_psnr(img,process_image(denoised))
    total_psnr_denoised +=psnr_denoised
    counter+=1
    print(counter)
print("Average GT to Noisy PSNR: " +str(total_psnr/counter))
print("Average Denoising Function PSNR: " +str(total_psnr_denoised/counter))

320
Average GT to Noisy PSNR: 24.843657291388904
Average Denoising Function PSNR: 27.646683580850027
